In [28]:
# Importation des bibliothèques nécessaires
from keras.preprocessing.image import img_to_array  # Convertir une image en tableau NumPy
from sklearn.utils import shuffle  # Mélanger les données
from sklearn.preprocessing import LabelBinarizer  # Convertir les étiquettes en format binaire
from sklearn.model_selection import train_test_split  # Diviser les données en ensembles d'entraînement et de test
from keras.preprocessing.image import ImageDataGenerator  # Génération d'images augmentées
from sklearn.metrics import classification_report, confusion_matrix  # Évaluation des performances du modèle
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img  # Traitement d'images avec Keras


In [2]:
train_data_dir = '/kaggle/input/agriculture-crop-images/kag2' 

**Data Augmentation Configuration and Training Data Generator Initialization**

In [3]:
# Configuration de l'augmentation des données d'entraînement
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,        # Mise à l'échelle des valeurs de pixel entre 0 et 1
    horizontal_flip=True,     # Flip horizontal aléatoire des images
    vertical_flip=True,       # Flip vertical aléatoire des images
    rotation_range=90         # Rotation aléatoire des images dans la plage de 0 à 90 degrés
)

# Création d'un générateur de lots d'entraînement à partir du répertoire des données
train_generator = train_datagen.flow_from_directory(
    train_data_dir,           # Chemin du répertoire d'entraînement
    target_size=(224, 224),    # Redimensionnement des images à la taille spécifiée
    batch_size=64,            # Taille du lot
    class_mode='categorical',  # Mode de catégorisation pour les étiquettes (one-hot encoding)
    shuffle=True               # Mélanger les données à chaque époque
)


Found 804 images belonging to 5 classes.


In [29]:
# Indices des classes dans le générateur de données d'entraînement
train_generator.class_indices


{'jute': 0, 'maize': 1, 'rice': 2, 'sugarcane': 3, 'wheat': 4}

**Model Training MobileNet**

In [31]:
# Importation d'une architecture de modèle pré-entraîné MobileNet et des modules nécessaires
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization

import matplotlib.pyplot as plt


**Initialization of MobileNet Pre-trained Model and Freezing Layers**

In [32]:
# Création d'un modèle pré-entraîné MobileNet avec des poids provenant d'ImageNet
pre_model = MobileNet(weights='imagenet', 
                      include_top=False, 
                      input_shape=(224, 224, 3))

# Geler toutes les couches du modèle pour les rendre non entraînables
for layer in pre_model.layers:
    layer.trainable = False
  

In [7]:
pre_model.summary()


Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32

**Adding Flatten and Final Dense Layer to MobileNet Pre-trained Model**

In [33]:
# Ajout d'une couche d'aplatissement (Flatten) à la sortie du modèle MobileNet pré-entraîné
last_layer = Flatten()(pre_model.output)

# Ajout d'une couche dense avec 5 neurones (correspondant au nombre de classes) et une activation softmax
final_layer = Dense(5, activation='softmax')(last_layer)


In [34]:
# Création d'un nouveau modèle en spécifiant les entrées (inputs) et les sorties (outputs)
model = Model(inputs=pre_model.input, outputs=final_layer)



In [10]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128   

**Compiling the Model**

In [35]:
# Compilation du modèle avec un optimiseur Adam, une fonction de perte categorical_crossentropy et la métrique accuracy
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
model.fit_generator(train_generator, epochs=10)


/tmp/ipykernel_42/2975305634.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10)


Epoch 1/10
13/13 [==============================] - 32s 2s/step - loss: 5.1995 - accuracy: 0.4515
Epoch 2/10
13/13 [==============================] - 28s 2s/step - loss: 1.6023 - accuracy: 0.7326
Epoch 3/10
13/13 [==============================] - 29s 2s/step - loss: 0.8511 - accuracy: 0.8246
Epoch 4/10
13/13 [==============================] - 28s 2s/step - loss: 0.6536 - accuracy: 0.8744
Epoch 5/10
13/13 [==============================] - 29s 2s/step - loss: 0.3258 - accuracy: 0.9192
Epoch 6/10
13/13 [==============================] - 28s 2s/step - loss: 0.4008 - accuracy: 0.9042
Epoch 7/10
13/13 [==============================] - 28s 2s/step - loss: 0.2505 - accuracy: 0.9341
Epoch 8/10
13/13 [==============================] - 28s 2s/step - loss: 0.1786 - accuracy: 0.9577
Epoch 9/10
13/13 [==============================] - 29s 2s/step - loss: 0.1191 - accuracy: 0.9627
Epoch 10/10
13/13 [==============================] - 29s 2s/step - loss: 0.0952 - accuracy: 0.9701


**Save Model**

In [36]:
# Sauvegarde du modèle sous le nom 'my_model.keras'
model.save('my_model.keras')


**Evaluating the model**

In [37]:
# Évaluation du modèle sur les données de test à l'aide du générateur
model.evaluate_generator(test_generator)


/tmp/ipykernel_42/2620695681.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(test_generator)


[2.4068655967712402, 0.15920397639274597]

**predictive system**

In [27]:
#TestData
test_data_dir = '/kaggle/input/agriculture-crop-images/crop_images'
test_datagen = ImageDataGenerator(rescale=1./255)
 
test_batchsize = 64
 
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(224,225),
        batch_size=64,
        class_mode='categorical',
        shuffle=False)

Found 201 images belonging to 5 classes.


In [25]:
import numpy as np
model.load_weights("my_model.keras")
class_labels = test_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
Y_pred = model.predict_generator(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

/tmp/ipykernel_42/119025590.py:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_generator)


In [18]:
import cv2
import numpy as np

In [23]:
def predict_crop(path,actual,class_labels):
    predict_datagen = ImageDataGenerator(rescale=1./255)
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    img = np.array(img).reshape((1, 224, 224, 3))
    Y_pred = model.predict(img)
    y_pred = np.argmax(Y_pred, axis=1)
    if y_pred == actual:
        print('Correct prediction')
    else:
        print("Wrong Prediction!!")
    print('Actual class "{0}" and predicted class "{1}"'.format(class_labels[int(y_pred)],class_labels[actual]))

**Some predictions**

In [20]:
predict_crop('/kaggle/input/agriculture-crop-images/test_crop_image/jute-field.jpg',0,class_labels)

1/1 [==============================] - 1s 626ms/step
Wronge Prediction!!
Actual class "maize" and predicted class "jute"


In [21]:
predict_crop('/kaggle/input/agriculture-crop-images/test_crop_image/maize_fieldmexico.jpeg',1,class_labels)

1/1 [==============================] - 0s 63ms/step
Correct prediction
Actual class "maize" and predicted class "maize"


In [22]:
predict_crop('/kaggle/input/agriculture-crop-images/test_crop_image/rice-field.jpg',0,class_labels)

1/1 [==============================] - 0s 56ms/step
Wronge Prediction!!
Actual class "maize" and predicted class "jute"


In [24]:
predict_crop('/kaggle/input/agriculture-crop-images/test_crop_image/sugarcane fields.jpg',1,class_labels)

1/1 [==============================] - 0s 59ms/step
Wrong Prediction!!
Actual class "rice" and predicted class "maize"
